In [1]:
%status

{'status': 'healthy',
 'startTime': 'Tue Aug 13 23:06:18 UTC 2024',
 'dbEngineVersion': '1.3.2.1.R1',
 'role': 'writer',
 'dfeQueryEngine': 'viaQueryHint',
 'gremlin': {'version': 'tinkerpop-3.7.1'},
 'sparql': {'version': 'sparql-1.1'},
 'opencypher': {'version': 'Neptune-9.0.20190305-1.0'},
 'labMode': {'ObjectIndex': 'disabled',
  'ReadWriteConflictDetection': 'enabled'},
 'features': {'SlowQueryLogs': 'disabled',
  'ResultCache': {'status': 'disabled'},
  'IAMAuthentication': 'disabled',
  'Streams': 'disabled',
  'AuditLog': 'disabled'},
 'settings': {'StrictTimeoutValidation': 'false',
  'clusterQueryTimeoutInMs': '120000',
  'SlowQueryLogsThreshold': '5000'}}

In [4]:
%%oc
MATCH (n)
DETACH DELETE n

In [5]:
%load

Button(description='Submit', style=ButtonStyle())

Output()

Clean data so that gFD is not violated

In [158]:
%%oc
MATCH (o:Order)
WITH o.customerID as id, COUNT(DISTINCT(o.shipName)) AS amount
WHERE amount > 1
RETURN id

In [160]:
%%oc
MATCH (o:Order{customerID: "ALFKI"})
SET o.shipName = "Alfred's Futterkise"

Execute DISTINCT query

In [161]:
%%oc
MATCH (o:Order) WHERE
EXISTS(o.customerID) AND
EXISTS(o.shipCity) AND
EXISTS(o.shipName) AND
EXISTS(o.shipPostalCode) AND
EXISTS(o.shipCountry) AND
EXISTS(o.shipAddress) AND
EXISTS(o.shipRegion)
RETURN DISTINCT(o.customerID)

Execute insertion of node including gFD verification (for existing equivalence class), Amazon Neptune does not support CALL procedures

In [74]:
%%oc
MERGE (new:Order{orderID: 99999, customerID: "SAVEA", shipCity: "Boise", shipName: "Save-a-lot Markets", shipPostalCode: 83720, shipCountry: "USA", shipAddress: "187 Suffolk Ln.", shipRegion: "ID"})
WITH new AS new_node
MATCH (o:Order) WHERE
EXISTS(o.customerID) AND
EXISTS(o.shipCity) AND
EXISTS(o.shipName) AND
EXISTS(o.shipPostalCode) AND
EXISTS(o.shipCountry) AND
EXISTS(o.shipAddress) AND
EXISTS(o.shipRegion)
WITH o.customerID AS ids, COUNT(DISTINCT(o.shipCity + o.shipName + o.shipPostalCode + o.shipCountry + o.shipAddress + o.shipRegion)) AS amount
WHERE amount > 1
RETURN ids, amount

Delete new node (return count to see that node deleted, not measuring time for this operation)

In [162]:
%%oc
MATCH (o:Order{orderID: 99999}) DELETE o
RETURN COUNT(o)

Execute insertion of node including gFD verification (including new equivalence class), Amazon Neptune does not support CALL procedures

In [163]:
%%oc
MERGE (new:Order{orderID: 99999, customerID: "new", shipCity: "new", shipName: "new", shipPostalCode: 99999, shipCountry: "new", shipAddress: "new", shipRegion: "new"})
WITH new AS new_node
MATCH (o:Order) WHERE
EXISTS(o.customerID) AND
EXISTS(o.shipCity) AND
EXISTS(o.shipName) AND
EXISTS(o.shipPostalCode) AND
EXISTS(o.shipCountry) AND
EXISTS(o.shipAddress) AND
EXISTS(o.shipRegion)
WITH o.customerID AS ids, COUNT(DISTINCT(o.shipCity + o.shipName + o.shipPostalCode + o.shipCountry + o.shipAddress + o.shipRegion)) AS amount
WHERE amount > 1
RETURN ids, amount

Delete new node (return count to see that node deleted, not measuring time for this operation)

In [164]:
%%oc
MATCH (o:Order{orderID: 99999}) DELETE o
RETURN COUNT(o)

Normalize graph: Update customer nodes (rather than creating new nodes) for each equivalence class

In [165]:
%%oc
MATCH (o:Order),(c:Customer) WHERE
EXISTS(c.customerID) AND
EXISTS(o.customerID) AND
(o.customerID = c.customerID) AND
EXISTS(o.shipCity) AND
EXISTS(o.shipName) AND
EXISTS(o.shipPostalCode) AND
EXISTS(o.shipCountry) AND
EXISTS(o.shipAddress) AND
EXISTS(o.shipRegion)
SET c.shipCity = o.shipCity
REMOVE o.shipCity
SET c.shipName = o.shipName
REMOVE o.shipName
SET c.shipPostalCode = o.shipPostalCode
REMOVE o.shipPostalCode
SET c.shipCountry = o.shipCountry
REMOVE o.shipCountry
SET c.shipAddress = o.shipAddress
REMOVE o.shipAddress
SET c.shipRegion = o.shipRegion
REMOVE o.shipRegion

Normalize graph: No need to create new edges as in this instance no new nodes have been created

Remove properties related to equivalence class from original nodes

In [166]:
%%oc
MATCH (o:Order) REMOVE o.customerID

Exectute in normalised graph

Execute DISTINCT query

In [167]:
%%oc
MATCH (c:Customer) WHERE
EXISTS(c.customerID) AND
EXISTS(c.shipCity) AND
EXISTS(c.shipName) AND
EXISTS(c.shipPostalCode) AND
EXISTS(c.shipCountry) AND
EXISTS(c.shipAddress) AND
EXISTS(c.shipRegion)
RETURN DISTINCT(c.customerID)

Execute insertion of node (for existing equivalence class)

In [199]:
%%oc
MATCH (c:Customer{customerID: 'SAVEA'}) MERGE (o:Order{orderID: 99999})<-[r:PURCHASED]-(c)

Delete new node(s) and edges if created (return count to see that delete was succesful, not measuring time for this operation)

In [200]:
%%oc
MATCH (o:Order{orderID: 99999}) OPTIONAL MATCH (c:Customer{customerID: 'new'}) DETACH DELETE o,c
RETURN COUNT(o), COUNT(c)

Execute insertion of node including (including new equivalence class node)

In [227]:
%%oc
MERGE (o:Order{orderID: 99999})<-[r:PURCHASED]-(c:Customer{customerID: 'new', shipCity: 'new', shipName: 'new', shipPostalCode: 99999, shipCountry: 'new', shipAddress: 'new', shipRegion: 'new'})

Delete new node(s) and edges if created (return count to see that delete was succesful, not measuring time for this operation)

In [228]:
%%oc
MATCH (o:Order{orderID: 99999}) OPTIONAL MATCH (c:Customer{customerID: 'new'}) DETACH DELETE o,c
RETURN COUNT(o), COUNT(c)